In [4]:
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, Accuracy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam,schedules
from sklearn.metrics import confusion_matrix
import numpy as np




In [5]:
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import random
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
TRAIN_SPLIT = 0.85

DIR = './'
imgH = 32
imgW = 32

# file = open('data_index.txt', 'w')
count = 0

def load_image_data():
    imgDir = DIR + 'Normal/'
    imgSet1 = prepare_image_array(imgDir, imgW, imgH)
    m = imgSet1.shape[0]

    imgDir = DIR + 'OverExposed/'
    imgSet2 = prepare_image_array(imgDir, imgW, imgH)
    n = imgSet2.shape[0]

    imgDir = DIR + 'UnderExposed/'
    imgSet3 = prepare_image_array(imgDir, imgW, imgH)
    o = imgSet3.shape[0]

    # Put all image data into one array.
    imgSet = np.concatenate((imgSet1, imgSet2, imgSet3), axis=0)
    print(imgSet.shape)

    labelSet1 = np.zeros(m, dtype=np.uint8)
    labelSet2 = np.ones(n, dtype=np.uint8)
    label_o = np.ones(o, dtype=np.uint8)
    labelSet3 = np.add(label_o, label_o)
    labelSet = np.concatenate((labelSet1, labelSet2, labelSet3), axis=0)


    p = imgSet.shape[0]  # p = n + m + o

    # random.shuffle(indices)
    # for i in range(len(indices)):
    #     file.write(str(indices[i])+",")
    
    # file.close()
    f = open('data_index.txt', 'r')
    indices = []
    for x in f.read().split(','):
        if len(x) > 0:
            num = ''
            for j in x:
                if(j>='0' and j<='9'):
                    num +=j;
            if len(num)>0 and  int(num)<p:
                indices.append(int(num)) 
   
    # print(indices)
    imgSet = imgSet[indices]
    labelSet = labelSet[indices]
    print(labelSet[:5])
    print("Label set : ")
    labelSet = labelSet.reshape(p, 1)

#   one hot encoding

    labelSet = to_categorical(labelSet)

    # print(labelSet[:5])
    
    # print(imgSet[0])
    imgSet = imgSet.astype(np.float32)
    imgSet = imgSet/255.0

    r = int(p * TRAIN_SPLIT)
    trainX = imgSet[:r]
    trainY = labelSet[:r]
    testX = imgSet[r:]
    testY = labelSet[r:]

    return trainX, trainY, testX, testY



def prepare_image_array(imgDir, imgW, imgH):
    imgList = os.listdir(imgDir)
    # print(imgList)
    n = len(imgList)

    imgSet = []
    for i in range(n):
        imgPath = imgDir + imgList[i]
        if (os.path.exists(imgPath)):
            # print(imgPath)
            img = cv2.imread(imgPath)
            resizedImg = cv2.resize(img, (imgW, imgH))
            rgbImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
            imgSet.append(rgbImg)
        else:
            print("It is not a valid image path.")

    # print("total image "+str(len(imgSet)))
    imgSet = np.array(imgSet, dtype=np.uint8)
    # print("total shape "+str(imgSet.shape))

    return imgSet



In [6]:
train_x,train_Y, testInSet, testOutSet =load_image_data()


(8984, 32, 32, 3)
[0 2 1 2 0]
Label set : 


In [7]:
from sklearn.metrics import classification_report

def run_model(modelpath,testInSet, testOutSet):

    lr_schedule = schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=10000,
        decay_rate=0.9)

    sgd = SGD(learning_rate=lr_schedule)


    model = load_model(modelpath, compile = True)
    model.compile(loss = 'mse', optimizer = sgd, metrics = ['accuracy', Recall(), Precision()])
    


#     y_test = np.argwhere(testOutSet)[:,1]
    # print(y_test)
#     indices = np.argwhere(y_test == 0).flatten()
#     print(indices.shape)
#     testInNormal = testInSet[indices]
#     testOutNormal = testOutSet[indices]



#     indices = np.argwhere(y_test == 1).flatten()
#     testInOver = testInSet[indices]
#     testOutOver = testOutSet[indices]
#     indices = np.argwhere(y_test == 2).flatten()
#     testInUnder = testInSet[indices]
#     testOutUnder = testOutSet[indices]



    '''	Test Model. '''
    lossT, accT, precisionT, recallT = model.evaluate(testInSet, testOutSet)
#     lossN, accN, precisionN, recallN = model.evaluate(testInNormal, testOutNormal)
#     lossO, accO, precisionO, recallO = model.evaluate(testInOver, testOutOver)
#     lossU, accU, precisionU, recallU = model.evaluate(testInUnder, testOutUnder)


    print("\n\nlossT = %s, accT = %s,  precisionT = %s, recallT = %s "%(lossT, accT, precisionT, recallT))
#     print("lossN = %s, accN = %s,  precisionN = %s, recallN = %s "%(lossN, accN, precisionN, recallN))
#     print("lossO = %s, accO = %s,  precisionO = %s, recallO = %s "%(lossO, accO, precisionO, recallO))
#     print("lossU = %s, accU = %s,  precisionU = %s, recallU = %s "%(lossU, accU, precisionU, recallU))

    predicted_output = []
    prediction = model.predict(testInSet)
    for i in prediction:
        predicted_output.append(np.argmax(i)) 

    actual_output = []
    for i in testOutSet:
        actual_output.append(np.argmax(i)) 
    con_mat = confusion_matrix(actual_output, predicted_output)
#     for i in con_mat:
#         print(i)
        
    cm =  classification_report(actual_output,predicted_output )
#     print(cm)
    cm = cm.split(" ")
    count = 0
    cnt = 0
    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "
    acc = "Accuracy & \multicolumn{3}{r}{"
    mac = "Macro Avg "
    weight = "Weighted Avg "
    for i in cm:
        if len(i)==4:
            count = 0
            for j in i:
                if j=='.':
                    count =1
            if count ==1:
                if(cnt<3):
                    first_line +=" & %s"%i[2:]
                    first_line +='\%'
                if cnt>=3 and cnt<6:
                    second_line += " & %s"%i[2:]
                    second_line +='\%'
                if cnt>=6 and  cnt<9:
                    third_line += " & %s"%i[2:]
                    third_line +='\%'
                if cnt>=9 and cnt<10:
                    acc += " %s"%i[2:]
                    acc +='\%}'
                if cnt>=10 and cnt<13:
                    mac +=" & %s"%i[2:]
                    mac +='\%'
                if cnt>=13 and cnt<16:
                    weight += " & %s"%i[2:]
                    weight +='\%'
                cnt+=1

    print(first_line+'\\\\')
    print(second_line+'\\\\')
    print(third_line+'\\\\\\hline')
    print(acc+'\\\\')
    print(mac+'\\\\')
    print(weight+'\\\\\hline\hline\\thinlines')



    print("\n\n\n\n")


    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "

    count = 0
    for i in con_mat:
        if count==0:
            first_line+= ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==1:
            second_line+=  ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==2:
            third_line+=   ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        count+=1


    print(first_line+" \\\\\hline")
    print(second_line+" \\\\\hline")
    print(third_line+" \\\\\hline")

    
    return cm,con_mat

    

In [8]:
print("Non trainable")
DIR = './non-trainable/'
modelpath = DIR + 'VGG_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Non trainable
43/43 [==============================] - 7s 112ms/step - loss: 0.2218 - accuracy: 0.3197 - recall: 0.0000e+00 - precision: 0.0000e+00


lossT = 0.22179409861564636, accT = 0.3197329342365265,  precisionT = 0.0, recallT = 0.0 
Normal  & 32\% & 99\% & 48\%\\
Over Expsoed  & 50\% & 00\% & 00\%\\
Under Expsoed  & 52\% & 03\% & 06\%\\\hline
Accuracy & \multicolumn{3}{r}{ 32\%}\\
Macro Avg  & 44\% & 34\% & 18\%\\
Weighted Avg  & 45\% & 32\% & 17\%\\\hline\hline\thinlines





Normal  & 416 & 1 & 4 \\\hline
Over Expsoed  & 442 & 1 & 9 \\\hline
Under Expsoed  & 461 & 0 & 14 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.32',
  '',
  '',
  '',
  '',
  '',
  '0.99',
  '',
  '',
  '',
  '',
  '',
  '0.48',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.50',
  '',
  '',
  '',
  '',
  '',
  '0.00',
  '',
  '',
  '',
  '',
  '',
  '0.00',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.52',
  '',
  '',
  '',
  '',
  '',
  '0.03',
  '',
  '',
  '',
  '',
  '',
  '0.06',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [9]:
print("trainable")
DIR = './trainable/'
modelpath = DIR + 'trainable-tf-Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

trainable
43/43 [==============================] - 7s 109ms/step - loss: 0.1256 - accuracy: 0.7359 - recall_1: 0.6869 - precision_1: 0.7717


lossT = 0.12559440732002258, accT = 0.7359050512313843,  precisionT = 0.6869435906410217, recallT = 0.7716666460037231 
Normal  & 60\% & 59\% & 59\%\\
Over Expsoed  & 80\% & 75\% & 77\%\\
Under Expsoed  & 80\% & 85\% & 82\%\\\hline
Accuracy & \multicolumn{3}{r}{ 74\%}\\
Macro Avg  & 73\% & 73\% & 73\%\\
Weighted Avg  & 73\% & 74\% & 73\%\\\hline\hline\thinlines





Normal  & 248 & 85 & 88 \\\hline
Over Expsoed  & 99 & 339 & 14 \\\hline
Under Expsoed  & 68 & 2 & 405 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.60',
  '',
  '',
  '',
  '',
  '',
  '0.59',
  '',
  '',
  '',
  '',
  '',
  '0.59',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.80',
  '',
  '',
  '',
  '',
  '',
  '0.75',
  '',
  '',
  '',
  '',
  '',
  '0.77',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.80',
  '',
  '',
  '',
  '',
  '',
  '0.85',
  '',
  '',
  '',
  '',
  '',
  '0.82',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [10]:
print("fully connected ")
DIR = './Fully-Connnected-model/'
modelpath = DIR + 'Fully_connected_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

fully connected 
43/43 [==============================] - 2s 9ms/step - loss: 0.1114 - accuracy: 0.7678 - recall_2: 0.7522 - precision_2: 0.7836


lossT = 0.11143425852060318, accT = 0.7678041458129883,  precisionT = 0.7522255182266235, recallT = 0.7836167216300964 
Normal  & 67\% & 59\% & 62\%\\
Over Expsoed  & 83\% & 83\% & 83\%\\
Under Expsoed  & 79\% & 87\% & 83\%\\\hline
Accuracy & \multicolumn{3}{r}{ 77\%}\\
Macro Avg  & 76\% & 76\% & 76\%\\
Weighted Avg  & 76\% & 77\% & 76\%\\\hline\hline\thinlines





Normal  & 247 & 78 & 96 \\\hline
Over Expsoed  & 64 & 374 & 14 \\\hline
Under Expsoed  & 60 & 1 & 414 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.67',
  '',
  '',
  '',
  '',
  '',
  '0.59',
  '',
  '',
  '',
  '',
  '',
  '0.62',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.83',
  '',
  '',
  '',
  '',
  '',
  '0.83',
  '',
  '',
  '',
  '',
  '',
  '0.83',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.79',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '0.83',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [11]:
print("smalll")
DIR = './Small-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Small_CNN_Classifier.hdf5'

cm,con_mat = run_model(modelpath,testInSet, testOutSet)




smalll
43/43 [==============================] - 2s 7ms/step - loss: 0.1125 - accuracy: 0.7693 - recall_3: 0.7522 - precision_3: 0.7830


lossT = 0.11253418028354645, accT = 0.7692878246307373,  precisionT = 0.7522255182266235, recallT = 0.7830115556716919 
Normal  & 65\% & 64\% & 65\%\\
Over Expsoed  & 84\% & 81\% & 82\%\\
Under Expsoed  & 81\% & 84\% & 83\%\\\hline
Accuracy & \multicolumn{3}{r}{ 77\%}\\
Macro Avg  & 77\% & 77\% & 77\%\\
Weighted Avg  & 77\% & 77\% & 77\%\\\hline\hline\thinlines





Normal  & 271 & 71 & 79 \\\hline
Over Expsoed  & 74 & 365 & 13 \\\hline
Under Expsoed  & 74 & 0 & 401 \\\hline


In [12]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Large-Size-Convolution-Maxpooling-model
43/43 [==============================] - 3s 40ms/step - loss: 0.2189 - accuracy: 0.6981 - recall_4: 0.0000e+00 - precision_4: 0.0000e+00


lossT = 0.2188725471496582, accT = 0.6980712413787842,  precisionT = 0.0, recallT = 0.0 
Normal  & 57\% & 64\% & 60\%\\
Over Expsoed  & 84\% & 64\% & 72\%\\
Under Expsoed  & 72\% & 81\% & 76\%\\\hline
Accuracy & \multicolumn{3}{r}{ 70\%}\\
Macro Avg  & 71\% & 69\% & 70\%\\
Weighted Avg  & 71\% & 70\% & 70\%\\\hline\hline\thinlines





Normal  & 269 & 52 & 100 \\\hline
Over Expsoed  & 116 & 288 & 48 \\\hline
Under Expsoed  & 88 & 3 & 384 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.57',
  '',
  '',
  '',
  '',
  '',
  '0.64',
  '',
  '',
  '',
  '',
  '',
  '0.60',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.84',
  '',
  '',
  '',
  '',
  '',
  '0.64',
  '',
  '',
  '',
  '',
  '',
  '0.72',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.72',
  '',
  '',
  '',
  '',
  '',
  '0.81',
  '',
  '',
  '',
  '',
  '',
  '0.76',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

###### 

In [ ]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)a  = {}